### --- Day 6: Chronal Coordinates ---
- Get the Points (call them destinations)
- From origin {0,0}, sort list into an ordereddict
- Define grid from the lowest to highest distinations from origin
- Create an object to represent each node that contains a destination flag and an array of nearest neighbors
- Iterate entire grid finding nearest destination to each point using Manhatten distance formula (if x=(a,b) and y=(c,d), then |a−c|+|b−d|)
- iterate entire grid, for each position add all unique nearest neighbors to a hashmap.
- display largest value

In [1]:
class Coordinate():
    def __init__(self, line):
        x = line.split(",")[0].strip()
        y = line.split(",")[1].strip()
        self.x = int(x)
        self.y = int(y)
    def __str__(self):
        return "[{0},{1}]".format(self.x,self.y)
    def __repr__(self):
        return self.__str__()

val1 = Coordinate("0,0")
val2 = Coordinate("1,1")
print(val1)
print(val2)

[0,0]
[1,1]


In [2]:
coords = open("test_input.txt", "r").readlines()
print(coords)

['1, 1\n', '1, 6\n', '8, 3\n', '3, 4\n', '5, 5\n', '8, 9']


In [26]:
def getManhattenDistanceBetween(val1,val2):
    if (not isinstance(val1,Coordinate) and not isinstance(val2,Coordinate)):
        print("Invalid input")
        return 
    res =  abs(val1.x-val2.x) + abs(val1.y-val2.y)
    if debug: print(" - abs({0}-{1}) + abs({2}-{3}) == {4}".format(val1.x,val2.x,val1.y,val2.y,res))
    return res

getManhattenDistanceBetween(Coordinate("0,0"),Coordinate("2, 2\n"))

 - abs(0-2) + abs(0-2) == 4


4

In [4]:
def name_destinations(coords):
    val = 65 
    results = []
    for coord in coords:
        c = Coordinate(coord)
        result = (chr(val), c)
        results.append(result)
        val += 1
        
    return results

destinations = name_destinations(coords)
print(destinations)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]


In [5]:
for d in destinations:
    print(d)
    
x_max = max(p[1].x for p in destinations)
y_max = max(p[1].y for p in destinations)
print("x_max: " + str(x_max))
print("y_max: " + str(y_max))

('A', [1,1])
('B', [1,6])
('C', [8,3])
('D', [3,4])
('E', [5,5])
('F', [8,9])
x_max: 8
y_max: 9


In [6]:
from pandas import * #! pip3 install pandas

x_max = 12 #max(p[1].x for p in destinations) + 1 #zero based
y_max = 20 #max(p[1].y for p in destinations) + 1 #zero based

dest_map = DataFrame([[0 for y in range(x_max)] for i in range(y_max)])

for name, coord in destinations:
    dest_map.loc[coord.y,coord.x] =  name     #faster, treats as one call

print(destinations)    
print(dest_map)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]
    0  1   2  3   4  5   6   7  8   9   10  11
0    0  0   0  0   0  0   0   0  0   0   0   0
1    0  A   0  0   0  0   0   0  0   0   0   0
2    0  0   0  0   0  0   0   0  0   0   0   0
3    0  0   0  0   0  0   0   0  C   0   0   0
4    0  0   0  D   0  0   0   0  0   0   0   0
5    0  0   0  0   0  E   0   0  0   0   0   0
6    0  B   0  0   0  0   0   0  0   0   0   0
7    0  0   0  0   0  0   0   0  0   0   0   0
8    0  0   0  0   0  0   0   0  0   0   0   0
9    0  0   0  0   0  0   0   0  F   0   0   0
10   0  0   0  0   0  0   0   0  0   0   0   0
11   0  0   0  0   0  0   0   0  0   0   0   0
12   0  0   0  0   0  0   0   0  0   0   0   0
13   0  0   0  0   0  0   0   0  0   0   0   0
14   0  0   0  0   0  0   0   0  0   0   0   0
15   0  0   0  0   0  0   0   0  0   0   0   0
16   0  0   0  0   0  0   0   0  0   0   0   0
17   0  0   0  0   0  0   0   0  0   0   0   0
18   0  0   0  0   0  

In [7]:
debug = True
def closest_match(position):
    distance = 1000
    match_destination = None
    for destination in destinations:
        tmp = getManhattenDistanceBetween(position,destination[1])
        
        if debug: print("{2}::{0}->{1}".format(destination[1],tmp,destination[0]))
        
        if not match_destination is None:
            if tmp<distance:                     #least distance
                distance = tmp
                match_destination = destination  #multiple match
            elif tmp==distance:
                match_destination = "."
            else:
                next                             #stop looking
            
        else:
            match_destination = destination[0]
            distance = tmp
    return match_destination 
        
dest = closest_match(Coordinate("1,4"))   
print("Closest destination to [1,4]: " + str(dest))

A::[1,1]->3
B::[1,6]->2
C::[8,3]->8
D::[3,4]->2
E::[5,5]->5
F::[8,9]->12
Closest destination to [1,4]: .


In [8]:
debug = False

def buildMap(destinations):
    x_max = max(p[1].x for p in destinations) + 1 #zero based
    y_max = max(p[1].y for p in destinations) + 1 #zero based
    dest_map = DataFrame([[0 for y in range(x_max)] for i in range(y_max)])

    print("Total size: {0}".format(len(dest_map)))
    for row_index, row in dest_map.iterrows():
        for col_index, val in enumerate(row):
            if val == 0:
                addr = "{0},{1}".format(col_index,row_index)
                dest = closest_match(Coordinate(addr)) 
                if debug: print("closest match to {0} is {1}".format(addr,dest))
                dest_map.loc[row_index,col_index] = dest[0].lower()
            else:
                next
        print(" - row {0}".format(row_index))
    
    return dest_map

dest_map = buildMap(destinations)
print(dest_map)

Total size: 10
 - row 0
 - row 1
 - row 2
 - row 3
 - row 4
 - row 5
 - row 6
 - row 7
 - row 8
 - row 9
   0  1  2  3  4  5  6  7  8
0  a  a  a  a  a  .  c  c  c
1  a  a  a  a  a  .  c  c  c
2  a  a  a  d  d  e  c  c  c
3  a  a  d  d  d  e  c  c  c
4  .  .  d  d  d  e  e  c  c
5  b  b  .  d  e  e  e  e  c
6  b  b  b  .  e  e  e  e  .
7  b  b  b  .  e  e  e  f  f
8  b  b  b  .  e  e  f  f  f
9  b  b  b  .  f  f  f  f  f


In [9]:
def mapResults(input_map):
    dict = {}
    for i, row in input_map.iterrows():
        for j, val in row.iteritems():
            if not val == '.':
                if val.lower() in dict:
                    dict[val.lower()] += 1
                else:
                    dict[val.lower()] = 1
    return dict
        
results = mapResults(dest_map)
print(results)


{'a': 15, 'c': 15, 'd': 9, 'e': 17, 'b': 14, 'f': 10}


## remove boundaries
Need to remove the infinite boundary values. To do this, take the first and last row of the map in both x and y dimensions, select all values and then remove those values from the map. What remains are the possible answers to this question.

In [10]:
def append_unique_values(list2append,list2check):
    for i in list2check:
        if not i in list2append:
            list2append.append(i)

In [11]:
print(results)
print(dest_map)

boundary_destination_list = []
append_unique_values(boundary_destination_list,dest_map.iloc[0])
append_unique_values(boundary_destination_list,dest_map.iloc[-1])
append_unique_values(boundary_destination_list,dest_map.iloc[:,0])
append_unique_values(boundary_destination_list,dest_map.iloc[:,-1])
print(boundary_destination_list)

{'a': 15, 'c': 15, 'd': 9, 'e': 17, 'b': 14, 'f': 10}
   0  1  2  3  4  5  6  7  8
0  a  a  a  a  a  .  c  c  c
1  a  a  a  a  a  .  c  c  c
2  a  a  a  d  d  e  c  c  c
3  a  a  d  d  d  e  c  c  c
4  .  .  d  d  d  e  e  c  c
5  b  b  .  d  e  e  e  e  c
6  b  b  b  .  e  e  e  e  .
7  b  b  b  .  e  e  e  f  f
8  b  b  b  .  e  e  f  f  f
9  b  b  b  .  f  f  f  f  f
['a', '.', 'c', 'b', 'f']


In [12]:
def remove_boundaries():
    boundary_destination_list = []
    append_unique_values(boundary_destination_list,dest_map.iloc[0])
    append_unique_values(boundary_destination_list,dest_map.iloc[-1])
    append_unique_values(boundary_destination_list,dest_map.iloc[:,0])
    append_unique_values(boundary_destination_list,dest_map.iloc[:,-1])
    
    non_boundary_results = {}
    for destination in results:
        if not destination in boundary_destination_list:
            non_boundary_results[destination] = results[destination]
        
    return non_boundary_results

print(results)
li = remove_boundaries()
print(li)   

{'a': 15, 'c': 15, 'd': 9, 'e': 17, 'b': 14, 'f': 10}
{'d': 9, 'e': 17}


In [13]:
def getMaxValue(res):
    max_key=''
    max_val=0
    for key,val in res.items():
        if val > max_val:
            max_val = val
            max_key = key
    print("Max Key is {0} :: {1}".format(max_key,max_val))

getMaxValue(li)

Max Key is e :: 17


In [14]:
coords = open("_6.txt", "r").readlines()
destinations = name_destinations(coords)

print("- Building Map")
dest_map = buildMap(destinations)
print("- Built Map")
print("- Mapping Results")
results = mapResults(dest_map)
print("- Removing Boundaries")
nonboundary_results = remove_boundaries()
getMaxValue(nonboundary_results)

FileNotFoundError: [Errno 2] No such file or directory: '_6.txt'

### --- Part Two ---

* for each given coordinate, find the total distance to all others. If that sum is less than a provided value, it is a desired region.
* add the size of all desired regions


In [ ]:
dest_map = None
dest_map = buildMap(destinations)
print(dest_map)

In [ ]:
getManhattenDistanceBetween(Coordinate("0,0"),Coordinate("1,6"))

In [29]:
debug = True
def sum_inrange_distances(cur, destinations, target_max=32):
    sum = 0
    for destination in destinations:
        if debug: print("Getting {destination} from {cur}".format(destination=destination[1],cur=cur))
        sum += getManhattenDistanceBetween(cur,destination[1])
    if debug: print(" - Sum: " + str(sum))
    if sum<target_max:
        return sum
    else:
        return 0

print(sum_inrange_distances(Coordinate("4,3"),destinations))

Getting [1,1] from [4,3]
 - abs(4-1) + abs(3-1) == 5
Getting [1,6] from [4,3]
 - abs(4-1) + abs(3-6) == 6
Getting [8,3] from [4,3]
 - abs(4-8) + abs(3-3) == 4
Getting [3,4] from [4,3]
 - abs(4-3) + abs(3-4) == 2
Getting [5,5] from [4,3]
 - abs(4-5) + abs(3-5) == 3
Getting [8,9] from [4,3]
 - abs(4-8) + abs(3-9) == 10
 - Sum: 30
30


In [35]:
debug = False
max_range = 32
count = 0
for i, row in dest_map.iterrows():
        for j, val in row.iteritems():
            pos = Coordinate("{0},{1}".format(i,j))
            res = sum_inrange_distances(pos,destinations)
            if res>0: count = count + 1
print (count)

16


In [39]:
debug = False
max_range = 10000
count = 0

coords = open("6.txt", "r").readlines()
destinations = name_destinations(coords)

print("- Building Map")
dest_map = buildMap(destinations)

for i, row in dest_map.iterrows():
        for j, val in row.iteritems():
            pos = Coordinate("{0},{1}".format(i,j))
            res = sum_inrange_distances(pos,destinations,max_range)
            if res>0: 
                count = count + 1
                print("Adding {0}".format(count))
print (count)

- Building Map
Total size: 357
 - row 0
 - row 1
 - row 2
 - row 3
 - row 4
 - row 5
 - row 6
 - row 7
 - row 8
 - row 9
 - row 10
 - row 11
 - row 12
 - row 13
 - row 14
 - row 15
 - row 16
 - row 17
 - row 18
 - row 19
 - row 20
 - row 21
 - row 22
 - row 23
 - row 24
 - row 25
 - row 26
 - row 27
 - row 28
 - row 29
 - row 30
 - row 31
 - row 32
 - row 33
 - row 34
 - row 35
 - row 36
 - row 37
 - row 38
 - row 39
 - row 40
 - row 41
 - row 42
 - row 43
 - row 44
 - row 45
 - row 46
 - row 47
 - row 48
 - row 49
 - row 50
 - row 51
 - row 52
 - row 53
 - row 54
 - row 55
 - row 56
 - row 57
 - row 58
 - row 59
 - row 60
 - row 61
 - row 62
 - row 63
 - row 64
 - row 65
 - row 66
 - row 67
 - row 68
 - row 69
 - row 70
 - row 71
 - row 72
 - row 73
 - row 74
 - row 75
 - row 76
 - row 77
 - row 78
 - row 79
 - row 80
 - row 81
 - row 82
 - row 83
 - row 84
 - row 85
 - row 86
 - row 87
 - row 88
 - row 89
 - row 90
 - row 91
 - row 92
 - row 93
 - row 94
 - row 95
 - row 96
 - row 97

Adding 738
Adding 739
Adding 740
Adding 741
Adding 742
Adding 743
Adding 744
Adding 745
Adding 746
Adding 747
Adding 748
Adding 749
Adding 750
Adding 751
Adding 752
Adding 753
Adding 754
Adding 755
Adding 756
Adding 757
Adding 758
Adding 759
Adding 760
Adding 761
Adding 762
Adding 763
Adding 764
Adding 765
Adding 766
Adding 767
Adding 768
Adding 769
Adding 770
Adding 771
Adding 772
Adding 773
Adding 774
Adding 775
Adding 776
Adding 777
Adding 778
Adding 779
Adding 780
Adding 781
Adding 782
Adding 783
Adding 784
Adding 785
Adding 786
Adding 787
Adding 788
Adding 789
Adding 790
Adding 791
Adding 792
Adding 793
Adding 794
Adding 795
Adding 796
Adding 797
Adding 798
Adding 799
Adding 800
Adding 801
Adding 802
Adding 803
Adding 804
Adding 805
Adding 806
Adding 807
Adding 808
Adding 809
Adding 810
Adding 811
Adding 812
Adding 813
Adding 814
Adding 815
Adding 816
Adding 817
Adding 818
Adding 819
Adding 820
Adding 821
Adding 822
Adding 823
Adding 824
Adding 825
Adding 826
Adding 827
Adding 828

Adding 1501
Adding 1502
Adding 1503
Adding 1504
Adding 1505
Adding 1506
Adding 1507
Adding 1508
Adding 1509
Adding 1510
Adding 1511
Adding 1512
Adding 1513
Adding 1514
Adding 1515
Adding 1516
Adding 1517
Adding 1518
Adding 1519
Adding 1520
Adding 1521
Adding 1522
Adding 1523
Adding 1524
Adding 1525
Adding 1526
Adding 1527
Adding 1528
Adding 1529
Adding 1530
Adding 1531
Adding 1532
Adding 1533
Adding 1534
Adding 1535
Adding 1536
Adding 1537
Adding 1538
Adding 1539
Adding 1540
Adding 1541
Adding 1542
Adding 1543
Adding 1544
Adding 1545
Adding 1546
Adding 1547
Adding 1548
Adding 1549
Adding 1550
Adding 1551
Adding 1552
Adding 1553
Adding 1554
Adding 1555
Adding 1556
Adding 1557
Adding 1558
Adding 1559
Adding 1560
Adding 1561
Adding 1562
Adding 1563
Adding 1564
Adding 1565
Adding 1566
Adding 1567
Adding 1568
Adding 1569
Adding 1570
Adding 1571
Adding 1572
Adding 1573
Adding 1574
Adding 1575
Adding 1576
Adding 1577
Adding 1578
Adding 1579
Adding 1580
Adding 1581
Adding 1582
Adding 1583
Addi

Adding 2378
Adding 2379
Adding 2380
Adding 2381
Adding 2382
Adding 2383
Adding 2384
Adding 2385
Adding 2386
Adding 2387
Adding 2388
Adding 2389
Adding 2390
Adding 2391
Adding 2392
Adding 2393
Adding 2394
Adding 2395
Adding 2396
Adding 2397
Adding 2398
Adding 2399
Adding 2400
Adding 2401
Adding 2402
Adding 2403
Adding 2404
Adding 2405
Adding 2406
Adding 2407
Adding 2408
Adding 2409
Adding 2410
Adding 2411
Adding 2412
Adding 2413
Adding 2414
Adding 2415
Adding 2416
Adding 2417
Adding 2418
Adding 2419
Adding 2420
Adding 2421
Adding 2422
Adding 2423
Adding 2424
Adding 2425
Adding 2426
Adding 2427
Adding 2428
Adding 2429
Adding 2430
Adding 2431
Adding 2432
Adding 2433
Adding 2434
Adding 2435
Adding 2436
Adding 2437
Adding 2438
Adding 2439
Adding 2440
Adding 2441
Adding 2442
Adding 2443
Adding 2444
Adding 2445
Adding 2446
Adding 2447
Adding 2448
Adding 2449
Adding 2450
Adding 2451
Adding 2452
Adding 2453
Adding 2454
Adding 2455
Adding 2456
Adding 2457
Adding 2458
Adding 2459
Adding 2460
Addi

Adding 3378
Adding 3379
Adding 3380
Adding 3381
Adding 3382
Adding 3383
Adding 3384
Adding 3385
Adding 3386
Adding 3387
Adding 3388
Adding 3389
Adding 3390
Adding 3391
Adding 3392
Adding 3393
Adding 3394
Adding 3395
Adding 3396
Adding 3397
Adding 3398
Adding 3399
Adding 3400
Adding 3401
Adding 3402
Adding 3403
Adding 3404
Adding 3405
Adding 3406
Adding 3407
Adding 3408
Adding 3409
Adding 3410
Adding 3411
Adding 3412
Adding 3413
Adding 3414
Adding 3415
Adding 3416
Adding 3417
Adding 3418
Adding 3419
Adding 3420
Adding 3421
Adding 3422
Adding 3423
Adding 3424
Adding 3425
Adding 3426
Adding 3427
Adding 3428
Adding 3429
Adding 3430
Adding 3431
Adding 3432
Adding 3433
Adding 3434
Adding 3435
Adding 3436
Adding 3437
Adding 3438
Adding 3439
Adding 3440
Adding 3441
Adding 3442
Adding 3443
Adding 3444
Adding 3445
Adding 3446
Adding 3447
Adding 3448
Adding 3449
Adding 3450
Adding 3451
Adding 3452
Adding 3453
Adding 3454
Adding 3455
Adding 3456
Adding 3457
Adding 3458
Adding 3459
Adding 3460
Addi

...

Adding 37864
Adding 37865
Adding 37866
Adding 37867
Adding 37868
Adding 37869
Adding 37870
Adding 37871
Adding 37872
Adding 37873
Adding 37874
Adding 37875
Adding 37876
Adding 37877
Adding 37878
Adding 37879
Adding 37880
Adding 37881
Adding 37882
Adding 37883
Adding 37884
Adding 37885
Adding 37886
Adding 37887
Adding 37888
Adding 37889
Adding 37890
Adding 37891
Adding 37892
Adding 37893
Adding 37894
Adding 37895
Adding 37896
Adding 37897
Adding 37898
Adding 37899
Adding 37900
Adding 37901
Adding 37902
Adding 37903
Adding 37904
Adding 37905
Adding 37906
Adding 37907
Adding 37908
Adding 37909
Adding 37910
Adding 37911
Adding 37912
Adding 37913
Adding 37914
Adding 37915
Adding 37916
Adding 37917
Adding 37918
Adding 37919
Adding 37920
Adding 37921
Adding 37922
Adding 37923
Adding 37924
Adding 37925
Adding 37926
Adding 37927
Adding 37928
Adding 37929
Adding 37930
Adding 37931
Adding 37932
Adding 37933
Adding 37934
Adding 37935
Adding 37936
Adding 37937
Adding 37938
Adding 37939
Adding 37940

Adding 39024
Adding 39025
Adding 39026
Adding 39027
Adding 39028
Adding 39029
Adding 39030
Adding 39031
Adding 39032
Adding 39033
Adding 39034
Adding 39035
Adding 39036
Adding 39037
Adding 39038
Adding 39039
Adding 39040
Adding 39041
Adding 39042
Adding 39043
Adding 39044
Adding 39045
Adding 39046
Adding 39047
Adding 39048
Adding 39049
Adding 39050
Adding 39051
Adding 39052
Adding 39053
Adding 39054
Adding 39055
Adding 39056
Adding 39057
Adding 39058
Adding 39059
Adding 39060
Adding 39061
Adding 39062
Adding 39063
Adding 39064
Adding 39065
Adding 39066
Adding 39067
Adding 39068
Adding 39069
Adding 39070
Adding 39071
Adding 39072
Adding 39073
Adding 39074
Adding 39075
Adding 39076
Adding 39077
Adding 39078
Adding 39079
Adding 39080
Adding 39081
Adding 39082
Adding 39083
Adding 39084
Adding 39085
Adding 39086
Adding 39087
Adding 39088
Adding 39089
Adding 39090
Adding 39091
Adding 39092
Adding 39093
Adding 39094
Adding 39095
Adding 39096
Adding 39097
Adding 39098
Adding 39099
Adding 39100

Adding 40136
Adding 40137
Adding 40138
Adding 40139
Adding 40140
Adding 40141
Adding 40142
Adding 40143
Adding 40144
Adding 40145
Adding 40146
Adding 40147
Adding 40148
Adding 40149
Adding 40150
Adding 40151
Adding 40152
Adding 40153
Adding 40154
Adding 40155
Adding 40156
Adding 40157
Adding 40158
Adding 40159
Adding 40160
Adding 40161
Adding 40162
Adding 40163
Adding 40164
Adding 40165
Adding 40166
Adding 40167
Adding 40168
Adding 40169
Adding 40170
Adding 40171
Adding 40172
Adding 40173
Adding 40174
Adding 40175
Adding 40176
Adding 40177
Adding 40178
Adding 40179
Adding 40180
Adding 40181
Adding 40182
Adding 40183
Adding 40184
Adding 40185
Adding 40186
Adding 40187
Adding 40188
Adding 40189
Adding 40190
Adding 40191
Adding 40192
Adding 40193
Adding 40194
Adding 40195
Adding 40196
Adding 40197
Adding 40198
Adding 40199
Adding 40200
Adding 40201
Adding 40202
Adding 40203
Adding 40204
Adding 40205
Adding 40206
Adding 40207
Adding 40208
Adding 40209
Adding 40210
Adding 40211
Adding 40212

Adding 41172
Adding 41173
Adding 41174
Adding 41175
Adding 41176
Adding 41177
Adding 41178
Adding 41179
Adding 41180
Adding 41181
Adding 41182
Adding 41183
Adding 41184
Adding 41185
Adding 41186
Adding 41187
Adding 41188
Adding 41189
Adding 41190
Adding 41191
Adding 41192
Adding 41193
Adding 41194
Adding 41195
Adding 41196
Adding 41197
Adding 41198
Adding 41199
Adding 41200
Adding 41201
Adding 41202
Adding 41203
Adding 41204
Adding 41205
Adding 41206
Adding 41207
Adding 41208
Adding 41209
Adding 41210
Adding 41211
Adding 41212
Adding 41213
Adding 41214
Adding 41215
Adding 41216
Adding 41217
Adding 41218
Adding 41219
Adding 41220
Adding 41221
Adding 41222
Adding 41223
Adding 41224
Adding 41225
Adding 41226
Adding 41227
Adding 41228
Adding 41229
Adding 41230
Adding 41231
Adding 41232
Adding 41233
Adding 41234
Adding 41235
Adding 41236
Adding 41237
Adding 41238
Adding 41239
Adding 41240
Adding 41241
Adding 41242
Adding 41243
Adding 41244
Adding 41245
Adding 41246
Adding 41247
Adding 41248

Adding 41862
Adding 41863
Adding 41864
Adding 41865
Adding 41866
Adding 41867
Adding 41868
Adding 41869
Adding 41870
Adding 41871
Adding 41872
Adding 41873
Adding 41874
Adding 41875
Adding 41876
Adding 41877
Adding 41878
Adding 41879
Adding 41880
Adding 41881
Adding 41882
Adding 41883
Adding 41884
Adding 41885
Adding 41886
Adding 41887
Adding 41888
Adding 41889
Adding 41890
Adding 41891
Adding 41892
Adding 41893
Adding 41894
Adding 41895
Adding 41896
Adding 41897
Adding 41898
Adding 41899
Adding 41900
Adding 41901
Adding 41902
Adding 41903
Adding 41904
Adding 41905
Adding 41906
Adding 41907
Adding 41908
Adding 41909
Adding 41910
Adding 41911
Adding 41912
Adding 41913
Adding 41914
Adding 41915
Adding 41916
Adding 41917
Adding 41918
Adding 41919
Adding 41920
Adding 41921
Adding 41922
Adding 41923
Adding 41924
Adding 41925
Adding 41926
Adding 41927
Adding 41928
Adding 41929
Adding 41930
Adding 41931
Adding 41932
Adding 41933
Adding 41934
Adding 41935
Adding 41936
Adding 41937
Adding 41938